In [1]:
!nvidia-smi

Wed Nov 20 11:14:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 1. Install packages

In [2]:
!pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118
!pip install mmcv==2.0.1 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0/index.html

!pip install -qq ftfy openmim future tensorboard scikit-learn pandas
!mim install mmengine
!git clone -b main https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 585.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89991 sha256=f9d464a2585b0522216a78698cc3d6f0a1b7f3055404e8dc609c48b8079cc626
  Stored in directory: /root/.cache/pip/wheels/27/2c/b6/3ed2983b1b44fe0dea1bb35234b09f2c22fb8ebb308679c922
Successfully built lit
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu121
    Uninstalling torchvision-0.20.1+cu121

In [ ]:
# Ensure that you run the notebook from the mmdetection directory
%cd /workspace/mmdetection

/workspace/mmdetection


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# 2. Setup the dataset

In [4]:
# Download dataset from Google Drive
!gdown 16T40TdpaB8VXohm50SySREwrzbuPcJBC
!mkdir -p data/images
!unzip -qqnd data/images images.zip
!mv Train.csv Test.csv SampleSubmission.csv data/

Downloading...
From (original): https://drive.google.com/uc?id=16T40TdpaB8VXohm50SySREwrzbuPcJBC
From (redirected): https://drive.google.com/uc?id=16T40TdpaB8VXohm50SySREwrzbuPcJBC&confirm=t&uuid=663072ab-3412-473c-aa39-cc39f4b02dad
To: /content/mmdetection/images.zip
100% 4.29G/4.29G [00:46<00:00, 92.6MB/s]


In [5]:
!ls data

images	SampleSubmission.csv  Test.csv	Train.csv


In [6]:
# Download model weights
!mkdir -p output
!gdown -O output/epoch_30.pth 1C1b0PkTJdPTx_8Xti60mHTkc84HDJXmC


Downloading...
From (original): https://drive.google.com/uc?id=1C1b0PkTJdPTx_8Xti60mHTkc84HDJXmC
From (redirected): https://drive.google.com/uc?id=1C1b0PkTJdPTx_8Xti60mHTkc84HDJXmC&confirm=t&uuid=c86e37f9-1917-4d77-80f5-b34b45289874
To: /content/mmdetection/output/epoch_30.pth
100% 2.68G/2.68G [00:36<00:00, 73.0MB/s]


# 3. Inference

In [11]:
import pandas as pd, numpy as np
import sys,os,shutil,gc,re,json,glob,math,time,random,warnings,logging

from tqdm import tqdm
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold,StratifiedGroupKFold
import sklearn.metrics as skm
from sklearn import preprocessing
import torch
from torch import nn
import torch.nn.functional as F
import cv2
from mmengine import Config
from mmengine.runner import Runner
import mmdet
from mmdet.apis import init_detector, inference_detector
import mmcv

N_SPLITS = 5
RANDOM_STATE = 41
FOLD=0

def fix_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
fix_seed(RANDOM_STATE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [9]:
DIR_DATA = 'data'
CLASSES = {'Trophozoite':0,'WBC':1}
df_test = pd.read_csv(f'{DIR_DATA}/Test.csv')
df_test['path'] = f'{DIR_DATA}/images/'+df_test.Image_ID
df_test

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,path
0,id_5n9ov0rr22.jpg,NaN,NaN,NaN,NaN,NaN,NaN,data/images/id_5n9ov0rr22.jpg
1,id_k6s11k3klo.jpg,NaN,NaN,NaN,NaN,NaN,NaN,data/images/id_k6s11k3klo.jpg
2,id_eftvcf9djb.jpg,NaN,NaN,NaN,NaN,NaN,NaN,data/images/id_eftvcf9djb.jpg
3,id_kc5tmoj1xa.jpg,NaN,NaN,NaN,NaN,NaN,NaN,data/images/id_kc5tmoj1xa.jpg
4,id_tirid4qt68.jpg,NaN,NaN,NaN,NaN,NaN,NaN,data/images/id_tirid4qt68.jpg
...,...,...,...,...,...,...,...,...
1173,id_btrtdkgk4r.jpg,NaN,NaN,NaN,NaN,NaN,NaN,data/images/id_btrtdkgk4r.jpg
1174,id_straufuobm.jpg,NaN,NaN,NaN,NaN,NaN,NaN,data/images/id_straufuobm.jpg
1175,id_nnurq35wvp.jpg,NaN,NaN,NaN,NaN,NaN,NaN,data/images/id_nnurq35wvp.jpg
1176,id_hdqd25rput.jpg,NaN,NaN,NaN,NaN,NaN,NaN,data/images/id_hdqd25rput.jpg


In [14]:
cfg_path = 'configs/ddq/ddq-detr-4scale_swinl_8xb2-30e_coco.py'
cfg = Config.fromfile(cfg_path)

data_dir = 'data'
img_prefix = 'images'

bs = 2
cfg.train_dataloader.batch_size = bs

cfg.work_dir = './output'
metainfo = {
    'classes': ('TRO','WBC'),
    'palette': [
        (128, 60, 30),
        (220, 20, 60),
    ]
}
cfg.dataset_type = 'CocoDataset'

num_classes = 2

cfg.num_classes = num_classes
cfg.model.bbox_head.num_classes = num_classes

model_path = 'output/epoch_30.pth'
model = init_detector(cfg, model_path, 'cuda:0')

Loads checkpoint by local backend from path: output/epoch_30.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: decoder.aux_reg_branches.0.0.weight, decoder.aux_reg_branches.0.0.bias, decoder.aux_reg_branches.0.2.weight, decoder.aux_reg_branches.0.2.bias, decoder.aux_reg_branches.0.4.weight, decoder.aux_reg_branches.0.4.bias, decoder.aux_reg_branches.1.0.weight, decoder.aux_reg_branches.1.0.bias, decoder.aux_reg_branches.1.2.weight, decoder.aux_reg_branches.1.2.bias, decoder.aux_reg_branches.1.4.weight, decoder.aux_reg_branches.1.4.bias, decoder.aux_reg_branches.2.0.weight, decoder.aux_reg_branches.2.0.bias, decoder.aux_reg_branches.2.2.weight, decoder.aux_reg_branches.2.2.bias, decoder.aux_reg_branches.2.4.weight, decoder.aux_reg_branches.2.4.bias, decoder.aux_reg_branches.3.0.weight, decoder.aux_reg_branches.3.0.bias, decoder.aux_reg_branches.3.2.weight, decoder.aux_reg_branches.3.2.bias, decoder.aux_reg_branches.3.4.weight, decoder.aux_reg

In [15]:
# Initialize an empty list to store the results for all images
all_data = []
class2id = {'Trophozoite':0,'WBC':1}
id2class = {0:'Trophozoite',1:'WBC'}

minconf = 0.05
for _,row in tqdm(df_test.iterrows(),total=len(df_test)):
    img = cv2.imread(row.path)
    result = inference_detector(model, img)
    boxes = result.pred_instances.bboxes.cpu().numpy()
    confidences = result.pred_instances.scores.cpu().numpy()
    classes = result.pred_instances.labels.cpu().numpy()

    ixs = confidences>=minconf
    boxes = boxes[ixs]
    classes = classes[ixs]
    confidences = confidences[ixs]

    if len(boxes)==0:
        # If no detections, add NEG as the class
        all_data.append({
            'Image_ID': row.Image_ID,
            'class': 'NEG',
            'confidence': 1.0,  # You can set this to a default value
            'ymin': 0,  # Default value (no detection)
            'xmin': 0,  # Default value (no detection)
            'ymax': 0,  # Default value (no detection)
            'xmax': 0   # Default value (no detection)
        })
    else:
        # Iterate through the results for this image
        for box, cls, conf in zip(boxes, classes, confidences):
            x1, y1, x2, y2 = box
            detected_class = id2class[int(cls)]

            # Add the result to the all_data list
            all_data.append({
                'Image_ID': row.Image_ID,
                'class': detected_class,
                'confidence': conf,
                'ymin': y1,
                'xmin': x1,
                'ymax': y2,
                'xmax': x2
            })

sub = pd.DataFrame(all_data)
sub.to_csv('Submission.csv',index=False)
sub

  0%|          | 0/1178 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 1178/1178 [14:22<00:00,  1.37it/s]


,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,id_5n9ov0rr22.jpg,Trophozoite,0.777340,1797.231323,1440.857544,1903.334106,1543.061523
1,id_5n9ov0rr22.jpg,Trophozoite,0.763668,1749.035522,2778.903320,1850.769897,2877.233643
2,id_5n9ov0rr22.jpg,Trophozoite,0.751405,1025.633545,2422.210693,1130.193970,2516.692871
3,id_5n9ov0rr22.jpg,WBC,0.745139,1571.249634,65.264252,1888.657349,306.802002
4,id_5n9ov0rr22.jpg,Trophozoite,0.737245,1618.146118,2311.380615,1720.412720,2405.322510
...,...,...,...,...,...,...,...
37548,id_btrtdkgk4r.jpg,NEG,1.000000,0.000000,0.000000,0.000000,0.000000
37549,id_straufuobm.jpg,NEG,1.000000,0.000000,0.000000,0.000000,0.000000
37550,id_nnurq35wvp.jpg,NEG,1.000000,0.000000,0.000000,0.000000,0.000000
37551,id_hdqd25rput.jpg,NEG,1.000000,0.000000,0.000000,0.000000,0.000000
